# Teil 3: Training und Visualisierung

Hier werden Sie ein neuronales Netzwerk auf den Bilddaten trainieren und testen. Stellen Sie Fragen und versuchen Sie, ein tieferes Verständnis von Faltungsnetzen und Pytorch zu entwickeln.

In [ ]:
"""
Dieser Code kopiert und importiert notwendige Dateien in die virtuelle Maschine von Colab.
"""
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/Criscraft/HackathonDigitaltag2022.git'
    os.chdir('HackathonDigitaltag2022')

In [ ]:
import sys
from tqdm import tqdm
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from models import Network
import utils

# Konstanten
DATA_TRAIN = 'data/train/'
DATA_TEST = 'data/val/'
MAX_EPOCH = 100
BATCH_SIZE = 64
TRAIN_LOG_PATH = 'train_log.dat'
TEST_LOG_PATH = 'test_log.dat'
#TODO: Ändern Sie NO_CUDA auf False, falls Sie Ihre GPU benutzen wollen
NO_CUDA = False
MODEL_FILE_NAME = 'saved_model.pt'

## 1. Vorbereitung des Netzwerks

Erstellen Sie ein Netzwerk und transferieren Sie es auf die CPU oder die GPU. Die Fehlerfunktion ```nn.CrossEntropyLoss()``` und der Optimierer (etwa ```torch.optim.SGD```) werden hier gewählt. Setzen Sie die Lernrate zunächst auf 0.01 und den Impuls (momentum) auf 0.9. Die Parameter des Modells ```model.parameters()``` werden dem Optimierer zur Verfügung gestellt. Der Optimierer wird nur diese Parameter modifizieren, um die Fehlerfunktion zu minimieren.

In [ ]:
use_cuda = not NO_CUDA and torch.cuda.is_available()
cuda_args = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

#Das Pytorch Device regelt, ob 'cpu' oder 'cuda' (also die GPU) genutzt wird
device = torch.device("cuda") if use_cuda else torch.device("cpu")

#Netzwerk erstellen und auf das Device transferieren
model = Network()
#model = models.resnet50(pretrained=True)
#model.fc = nn.Linear(512 * 4, 2)

model = model.to(device)

# Fehlerfunktion erstellen
criterion = nn.CrossEntropyLoss()

# Optimierer (stochastic gradient descent) erstellen
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

## 2. Vorbereitung der Daten

Die Trainings- und Validierungsdaten werden durch die Dataset Klasse geladen. Der Dataloader zieht Bilder aus dem Dataset und fügt sie zu Batches zusammen. Die Transformationen ```transforms.Resize```, ```transforms.ToTensor``` und ```transforms.Normalize``` werden mittels ```transforms.Compose``` zu einer Transformation zusammen gefügt. Die Transformation wird auf jedes zu ladende Bild angewendet.

In [ ]:
#Die Transformation formt das Eingabebild in einen Tensor um und normalisiert diesen.
norm_mean = (0.485, 0.456, 0.406) # ImageNet mean color
norm_std = (0.229, 0.224, 0.225) # ImageNet std 

transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=norm_mean, std=norm_std),
    ])

#Trainigsdatensatz
trainset = ImageFolder(DATA_TRAIN, transform=transformations)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

#Testdatensatz
testset = ImageFolder(DATA_TEST, transform=transformations)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE)

## 3. Training und Validierung

Die nächste Codezelle implementiert die Trainingsschleife, in der die Parameter des Netzwerks iterativ verbessert werden. Lassen Sie sich Zeit, den Ablauf genau zu verstehen. Der Code tut folgendes:

1. Mithilfe der Funktionen ```utils.init_log``` eine Logdatei für die Trainingsergebnisse erzeugen - einmal für die Trainingsdaten und noch einmal für die Validierungsdaten
2. In einer for-Schleife über die Epochen iterieren. Nutzen Sie die Funktion tqdm, um einen Ladebalken darzustellen.
3. In einer Schleife über die Minibatches der Trainingsdaten iterieren, mit den Minibatches einen Forward Pass durchführen, den Fehler berechnen und die Parameter des Netzwerkes bei einem Optimierungsschritt anpassen
4. Den aktuellen Fehler und die Genauigkeit auf den Trainingsdaten in die Logdatei schreiben mit ```utils.write_log```.
5. In einer Schleife über die Minibatches der Validierungsdaten iterieren, mit den Minibatches einen Forward Pass durchführen und den Fehler berechnen.
6. Den aktuellen Fehler und die Genauigkeit auf den Validierungsdaten in die Logdatei schreiben.
7. Wenn MAX_EPOCH erreicht ist und das Training beendet ist, das Netzwerk auf der Platte abspeichern. Wir werden Ihr trainiertes Netzwerk morgen noch einmal benötigen!

In [ ]:
#Die beiden Logdateien werden mit utils.init_log für das Training und das Testen erstellt
utils.init_log(TRAIN_LOG_PATH)
utils.init_log(TEST_LOG_PATH)

for epoch in tqdm(range(1, MAX_EPOCH + 1)):
    
    ### Training ###
    
    # Die Listen 'losses' und 'correct' dienen dem Speichern der Zwischenergebnisse.
    losses = []
    correct = []
    n_images = 0
    
    # WICHTIG! Das Netzwerk in den Trainingsmodus umschalten!
    model.train()
    
    for data, target in train_loader:
        # Nebenschleife über die Training Minibatches
        
        n_images += len(target)
        
        # Transferieren von data und target auf das Pytorch Device
        data = data.to(device)
        target = target.to(device)
        
        # Die gespeicherten Gradienten auf 0 setzen
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(data)
        
        # Fehler auf dem Batch berechnen
        loss_batch = criterion(outputs, target)

        # vorhergesage Labels bestimmmen und zählen, wie viele Labels korrekt vorausgesagt wurden.
        pred = outputs.argmax(1)
        correct_batch = (target == pred).sum()
        
        # Berechnung des Gradienten
        loss_batch.backward()
        
        # Optimierungsschritt
        optimizer.step()
        
        losses.append(loss_batch.item())
        correct.append(correct_batch.item())
        
    losses = np.array(losses)
    correct = np.array(correct)
    
    # mittleren Fehler und die mittlere Genauigkeit berechnen
    loss = losses.mean()
    accuracy = correct.sum() / n_images
    
    # Speicherung der Zwischenergebnisse
    utils.write_log(TRAIN_LOG_PATH, epoch, loss, accuracy)
    
    
    ### Validierung ###
    
   losses = []
    correct = []
    n_images = 0
    
    # WICHTIG! Das Netzwerk in den Testmodus umschalten!
    model.eval()
    
    # WICHTIG! Zur Validierung die Berechnung der Gradienten ausstellen!
    with torch.no_grad():
        for data, target in test_loader:
            # Nebenschleife über die Test Minibatches
            
            n_images += len(target)
            
            data = data.to(device)
            target = target.to(device)
            
            outputs = model(data)
            
            loss_batch = criterion(outputs, target)
            
           pred = outputs.argmax(1)
            
            correct_batch = (target == pred).sum()
            
            losses.append(loss_batch.item())
            correct.append(correct_batch.item())
        
    losses = np.array(losses)
    correct = np.array(correct)
    loss = losses.mean()
    accuracy = correct.sum() / n_images
    utils.write_log(TEST_LOG_PATH, epoch, loss, accuracy)

    # Plotten des mittleren Fehlers
    utils.plot_loss(TRAIN_LOG_PATH, TEST_LOG_PATH)
    # Plotten der mittleren Genauigkeit
    utils.plot_accuracy(TRAIN_LOG_PATH, TEST_LOG_PATH)

# Abspeichern des trainierten Netzwerks
torch.save(model.state_dict(), MODEL_FILE_NAME)
    
    

## 4. Jetzt sind Sie dran!

Hier ein paar Ideen, wie Sie Ihren Algorithmus verbessern können:

- Augmentieren Sie Ihre Trainingsdaten: Bildausschnitte, Skalierung, Kontraständerungen, Weißes Rauschen, Spiegelung, ... was hilft Ihnen bei diesem Problem weiter? Das Paket torchvision hat einige nützliche Funktionen zu bieten!
- Ändern Sie Ihre Netzwerkarchitektur: Machen Sie das Netzwerk tiefer, flacher, weiter oder schmaler. Wie wirken sich Ihre Modifikationen auf Überanpassung und Genauigkeit aus?
- Modifizieren Sie die Lernrate. Probieren Sie, die Lernrate nach einiger Zeit zu verringern. Gewinnen Sie dadurch an Genauigkeit?
- Benutzen Sie einen anderen Optimierer wie Adam.
- Nutzen Sie Regularisierung: Experimentieren Sie z.B. mit Momentum oder Weight Decay. 
- Nutzen Sie ein vortrainiertes Netzwerk
- Falls Sie noch nicht genug haben: Trainieren Sie mehrere Netzwerke (nach einander), bündeln Sie sie zu einem Ensemble und nutzen Sie für die Validierung ihre gemittelte Ausgabe. Was müssen Sie bei der Mittelung der Ausgaben beachten?